# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
from sklearn import linear_model
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    
    # Lets start by divinding the points belonging to
    # the right lane from those belonging to the left lane.
    right_x = []
    right_y = []
    left_x = []
    left_y = []

    offset = 50
    img_x = img.shape[1]
    img_y = img.shape[0]
    vertical_boundary = int(img_x / 2)
    for line in lines:
        for x1,y1,x2,y2 in line:
            # Here we make the split left/right using the
            # apex of the ROI polygon, in this case a triangle.
            # Color the lines according to the polygon apex,
            # green lines on the left side of the road and
            # red lines on the right side of the road.
            if x1 >= vertical_boundary:
                right_x.append(x1)
                right_x.append(x2)
                right_y.append(y1)
                right_y.append(y2)
                #print("Found line in (x1, y1) (x2, y2) -", (x1, y1), (x2, y2), " RIGHT")
                cv2.line(img, (x1, y1), (x2, y2), [255, 0, 0], thickness)
            else:
                left_x.append(x1)
                left_x.append(x2)
                left_y.append(y1)
                left_y.append(y2)
                #print("Found line in (x1, y1) (x2, y2) -", (x1, y1), (x2, y2), " LEFT")
                cv2.line(img, (x1, y1), (x2, y2), [0, 255, 0], thickness)

    # It can happed to have an empty array therefore,
    # before going for the Linear Regression, lets
    # check if the array has some values.
    if len(right_x) > 0:
        x = np.array(right_x).reshape(-1, 1)
        y = np.array(right_y).reshape(-1, 1)

        # Linear regression for the right lane line.
        reg = linear_model.LinearRegression().fit(x, y)
        x_0 = np.array([int(img_x/2)]).reshape(1, -1)
        y_0 = reg.predict(x_0)
        x_1 = np.array([img_x-offset]).reshape(1, -1)
        y_1 = reg.predict(x_1)
        #print("RIGHT slope:     ", reg.coef_[0])
        #print("RIGHT intercept: ", reg.intercept_)
        cv2.line(img, (x_0, y_0), (x_1, y_1), [0, 0, 255], 6*thickness)

    if len(left_x) > 0:
        x = np.array(left_x).reshape(-1, 1)
        y = np.array(left_y).reshape(-1, 1)

        # Linear regression for the left lane line.
        reg = linear_model.LinearRegression().fit(x, y)
        x_0 = np.array([0+offset]).reshape(1, -1)
        y_0 = reg.predict(x_0)
        x_1 = np.array([int(img_x/2)]).reshape(1, -1)
        y_1 = reg.predict(x_1)
        #print("LEFT slope:     ", reg.coef_[0])
        #print("LEFT intercept: ", reg.intercept_)
        cv2.line(img, (x_0, y_0), (x_1, y_1), [0, 0, 255], 6*thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

In [ ]:
# Here is my code

# 1. Transform the image to gray scale
def pipeline_grayscale(img):
    return grayscale(img)

gray = pipeline_grayscale(image)
plt.imshow(gray, cmap='gray')

In [ ]:
# 2. Apply Canny Edge detection
def pipeline_canny_edge(img):
    """Computes high_threshold using Otsu's algorithm
    and the low_threshold as 0.5*high_threshold, then
    applies Canny edge detection."""
    high_threshold, otsu_img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    low_threshold = 0.5 * high_threshold
    return canny(otsu_img, low_threshold, high_threshold)

edges = pipeline_canny_edge(gray)
plt.imshow(edges, cmap='gray')

In [ ]:
# 3. Apply Gaussian Blur
def pipeline_gauss(img):
    kernel_size = 5
    return gaussian_blur(img, kernel_size)

gauss = pipeline_gauss(edges)
plt.imshow(gauss, cmap='gray')

In [ ]:
# 4. Extraxt the Region Of Interest - ROI
#    The ROI is a triangle.
#
#    Note: As seen in "The Great Robot Race" documentary,
#    Stanley was using a 4 sides polygon which can
#    lead to better reults that the triangle polygon.
#    Need to be tested.
def pipeline_roi(img):
    offset = 60
    img_x = img.shape[1]
    img_y = img.shape[0]
    img_center_x = int(img_x/2)
    img_center_y = int(img_y/2)

    apex = [img_center_x, img_center_y + offset]
    left_bottom = [0+int(offset/2), img_y-offset]
    right_bottom = [img_x-int(offset/2), img_y-offset]

    vertices = np.array([apex, left_bottom, right_bottom])
    return region_of_interest(img, [vertices])

region = pipeline_roi(gauss)
plt.imshow(region, cmap='gray')

In [ ]:
# 5. Hough Transformation
def pipeline_hough(img):
    rho = 1                         # distance resolution in pixels of the Hough grid
    theta = np.pi/180               # angular resolution in radians of the Hough grid
    threshold = 50                  # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 100              # minimum number of pixels making up a line
    max_line_gap = 50               # maximum gap in pixels between connectable line segments
    return hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap)

hough = pipeline_hough(region)
plt.imshow(hough, cmap='gray')

In [ ]:
# Final step. Display the starting image with detected lines
def pipeline_result(hough_img, original_img):
    return weighted_img(hough_img, original_img)

result = pipeline_result(hough, image)
plt.imshow(result)

run your solution on all test_images and make copies into the test_images directory).

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

## Pipeline description

The pipeline used in the video processing is programmed in the `process_image` function.
In particular the pipeline performs the following steps:

  * `pipeline_grayscale` converts the original image in greyscale;
  * `pipeline_canny_edge` performs canny edge detection to highlight the shape of the objects;
  * `pipeline_gauss` runs a gaussian filter so to reduce the noise of the canny edged image;
  * `pipeline_roi` selects a triangle as region of interest. Using a four sides polygon
    can lead to better performances;
  * `pipeline_hough` performs the Hough transformation to figure the lines position.
    In order to have stable left/right lines, I used a linear
    regression model. In particular I splitted the Hough transformation
    points into two sets: left lane line and right lane line and consequently,
    I run two separated linear regression model. The resulting lines are drawn
    using the reg.predict() sklearn function.
  * `pipeline_result` the final step simply overlaps the original image with the annotated image.

In [ ]:
def process_image(img):
    # NOTE: The output you return should be a color
    # image (3 channel) for processing video below
    # Your pipeline here, return the final
    # output (image with lines are drawn on lanes)
    gray = pipeline_grayscale(img)
    canny = pipeline_canny_edge(gray)
    gauss = pipeline_gauss(canny)
    region = pipeline_roi(gauss)
    hough = pipeline_hough(region)
    result = pipeline_result(hough, img)
    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))